In [733]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [734]:
movies = pd.read_csv('IMDB_dataset.csv', encoding='utf-16', encoding_errors='ignore', delimiter=";")
movies.head()

,IMBD title ID,Original title,Release year,Genre,Duration,Country,Content Rating,Director,Unnamed: 8,Income,Votes,Score
0,tt0111161,The Shawshank Redemption,1995-02-10,Drama,142,USA,R,Frank Darabont,NaN,$ 28815245,2.278.845,9.3
1,tt0068646,The Godfather,09 21 1972,"Crime, Drama",175,USA,R,Francis Ford Coppola,NaN,$ 246120974,1.572.674,9.2
2,tt0468569,The Dark Knight,23 -07-2008,"Action, Crime, Drama",152,US,PG-13,Christopher Nolan,NaN,$ 1005455211,2.241.615,9.
3,tt0071562,The Godfather: Part II,1975-09-25,"Crime, Drama",220,USA,R,Francis Ford Coppola,NaN,"$ 4o8,035,783",1.098.714,"9,.0"
4,tt0110912,Pulp Fiction,1994-10-28,"Crime, Drama",,USA,R,Quentin Tarantino,NaN,$ 222831817,1.780.147,"8,9f"


In [735]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IMBD title ID   100 non-null    object 
 1   Original title  100 non-null    object 
 2   Release year    100 non-null    object 
 3   Genre           100 non-null    object 
 4   Duration        99 non-null     object 
 5   Country         100 non-null    object 
 6   Content Rating  77 non-null     object 
 7   Director        100 non-null    object 
 8   Unnamed: 8      0 non-null      float64
 9   Income          100 non-null    object 
 10  Votes           100 non-null    object 
 11  Score           100 non-null    object 
dtypes: float64(1), object(11)
memory usage: 9.6+ KB


In [736]:
movies.drop(columns=['Unnamed: 8'], inplace=True)
movies.dropna(subset=["IMBD title ID"], inplace=True)
movies["IMBD title ID"] = movies["IMBD title ID"].str.strip("tt00")
print(movies.loc[movies["IMBD title ID"].isna()])
movies["IMBD title ID"].unique()


Empty DataFrame
Columns: [IMBD title ID, Original title, Release year, Genre, Duration, Country, Content Rating, Director, Income, Votes, Score]
Index: []


array(['111161', '68646', '468569', '71562', '110912', '16726', '108052',
       '50083', '1375666', '137523', '10983', '120737', '60196', '133093',
       '167261', '80684', '99685', '73486', '816692', '114369', '102926',
       '76759', '120815', '120689', '317248', '245429', '118799',
       '6751668', '3865', '47478', '172495', '407887', '482571', '88763',
       '120586', '110413', '103064', '114814', '110357', '7286456',
       '1675434', '253474', '2582802', '54215', '34583', '64116', '95327',
       '95765', '27977', '1345836', '1853728', '209144', '91097', '81505',
       '82971', '4154756', '78748', '4154796', '78788', '364569', '57012',
       '47396', '2380307', '405094', '4633694', '1187043', '119698',
       '87843', '32553', '43014', '361748', '169547', '112573', '8619',
       '105236', '338013', '114709', '119217', '208092', '180093',
       '66921', '435761', '8625', '75314', '211915', '93058', '90605',
       '62622', '33467', '8579674', '86879', '52357', '53125', '2

In [737]:
movies["Duration"].unique()
movies["Duration"] = movies["Duration"].str.strip(" c-InfNot Applicable")
movies["Duration"].unique()
movies.loc[movies["Duration"].isin([""]), "Duration"] = np.nan
movies["Duration"].unique()
movies["Duration"] = movies["Duration"].astype(float)
movies["Duration"]=movies["Duration"].fillna(movies["Duration"].mean().round(0))
movies["Duration"] = movies["Duration"].astype(int)
movies["Duration"].unique()

array([142, 175, 152, 220, 135, 201,  96, 148, 178, 161, 179, 146, 169,
       127, 118, 121, 189, 130, 125, 116, 132, 207, 155, 151, 119, 110,
       137, 106,  88, 122, 112, 150, 109, 102, 165,  89,  87, 164, 113,
        98, 115, 149, 117, 181, 147, 120,  95, 105, 170, 134, 229, 153,
       131,  99, 108,  81, 126, 104, 136, 103, 114, 160, 128, 228, 129,
       123])

In [738]:
movies[movies.duplicated()] #no duplicates
movies["Release year"] = movies["Release year"].str.strip()
movies["Release year"] = movies["Release year"].str.replace("/","-")
movies["Release year"] = pd.to_datetime(movies["Release year"],errors = 'coerce').dt.date
mode = movies["Release year"].mode()[0]
movies["Release year"]=movies["Release year"].fillna(mode)
movies["Release year"].value_counts()

Release year
2000-05-19    13
1995-02-10     1
1975-09-25     1
1994-10-28     1
1994-03-11     1
              ..
1974-03-21     1
1982-03-18     1
1965-12-20     1
2011-10-21     1
1953-02-05     1
Name: count, Length: 88, dtype: int64

In [739]:
movies["Genre"].unique()
# Split the 'Genre' column by comma
split_genre = movies['Genre'].str.split(',', expand=True)

unique_values = pd.unique(split_genre.values.ravel())
unique_values = [val.strip() for val in unique_values if pd.notna(val)]
print(unique_values)

# Step 3: One-Hot Encoding (Create new columns for each unique value)
for val in unique_values:
    movies[val] = movies['Genre'].apply(lambda x: 1 if val in x else 0)

# Display the updated DataFrame
movies.drop(columns=['Genre'], inplace=True)
movies.head()

['Drama', 'Crime', 'Drama', 'Action', 'Crime', 'Adventure', 'Biography', 'History', 'Sci-Fi', 'Romance', 'Western', 'Fantasy', 'Adventure', 'Mystery', 'Thriller', 'War', 'Animation', 'Family', 'Comedy', 'Comedy', 'Music', 'Horror', 'Western', 'Mystery', 'Horror', 'Action', 'Film-Noir', 'Biography', 'Musical']


,IMBD title ID,Original title,Release year,Duration,Country,Content Rating,Director,Income,Votes,Score,...,Mystery,Thriller,War,Animation,Family,Comedy,Music,Horror,Film-Noir,Musical
0,111161,The Shawshank Redemption,1995-02-10,142,USA,R,Frank Darabont,$ 28815245,2.278.845,9.3,...,0,0,0,0,0,0,0,0,0,0
1,68646,The Godfather,2000-05-19,175,USA,R,Francis Ford Coppola,$ 246120974,1.572.674,9.2,...,0,0,0,0,0,0,0,0,0,0
2,468569,The Dark Knight,2000-05-19,152,US,PG-13,Christopher Nolan,$ 1005455211,2.241.615,9.,...,0,0,0,0,0,0,0,0,0,0
3,71562,The Godfather: Part II,1975-09-25,220,USA,R,Francis Ford Coppola,"$ 4o8,035,783",1.098.714,"9,.0",...,0,0,0,0,0,0,0,0,0,0
4,110912,Pulp Fiction,1994-10-28,135,USA,R,Quentin Tarantino,$ 222831817,1.780.147,"8,9f",...,0,0,0,0,0,0,0,0,0,0


In [740]:
movies.loc[movies["Country"].str.contains("US"), "Country"] = "USA"
movies.loc[movies["Country"].str.contains("Germany"), "Country"] = "Germany"
movies.loc[movies["Country"].str.contains("New Z"), "Country"] = "New Zealand"
movies.loc[movies["Country"].str.contains("Italy"), "Country"] = "Italy"
movies["Country"].unique()

array(['USA', 'New Zealand', 'UK', 'Italy', 'Brazil', 'Japan',
       'South Korea', 'France', 'Germany', 'India', 'Denmark', 'Iran'],
      dtype=object)

In [741]:
movies["Content Rating"].unique()
movies["Content Rating"] = movies["Content Rating"].fillna("Not Rated")
movies.loc[movies["Content Rating"].str.contains("Not Rated|Approved|Unrated"), "Content Rating"] = "Not Rated"
movies["Content Rating"] = movies["Content Rating"].astype("category")
movies["Content Rating"] = movies["Content Rating"].cat.set_categories(new_categories=["Not Rated", "G", "PG", "PG-13", "R"], ordered=True)
movies["Content Rating"].value_counts()

Content Rating
R            45
Not Rated    26
PG-13        12
PG           11
G             6
Name: count, dtype: int64

In [742]:
movies["Income"].unique()
movies["Income"]=movies["Income"].str.strip("$ ").str.replace(",","").str.replace("o","0").astype(int)
movies["Income"].unique()

array([  28815245,  246120974, 1005455211,  408035783,  222831817,
       1142271098,  322287794,        576,  869784991,  101218804,
        678229452,  887934303,   25252481,  465718588,  951227416,
        549265501,   46879633,  108997629,  696742056,  327333559,
        272753884,  775768912,  482349603,  286801374,   30680793,
        355467056,  230098753,  257604912,    6130720,     322773,
        465361176,  291465034,  109676311,  388774684,   23875127,
         19552639,  520884847,   23341568,  968511805, 1074251311,
        426588510,  120072577,   48983260,   32008644,    4374761,
           112911,     516962,   13826605,     457688, 1081133191,
        425368238,   39970386,  521311860,   46520613,  390133212,
       2048359754,  108110316, 2797800564,   91968688,   15002116,
          9443876,   37032034,  807083670,   77356942,  375540831,
         60262836,  169785629,    5472914,     969879,     299645,
        321455689,  356296601,  213216216,  475347111,    2889

In [743]:
movies["Votes"].unique()
movies["Votes"]=movies["Votes"].str.replace(".","").astype(int)
movies["Votes"].unique()

array([2278845, 1572674, 2241615, 1098714, 1780147, 1604280, 1183248,
        668473, 2002816, 1807440, 1755490, 1619920,  672499, 1632315,
       1449778, 1132073,  991505,  891071, 1449256, 1402015, 1234134,
       1204107, 1203825, 1112336,  685856,  626693,  605648,  470931,
        388310,  307958, 1308193, 1159703, 1155723, 1027330, 1014218,
       1007598,  974970,  968947,  917248,  855097,  736691,  707942,
        690732,  586765,  509953,  295220,  225438,  223050,  211250,
       1480582, 1317856, 1098879,  974734,  869480,  865510,  796486,
        768874,  754786,  591251,  501082,  441115,  432390,  352455,
        349642,  335892,  332217,  331045,  302317,  197381,  195789,
       1229958, 1049009,  941683,  928036,  896551,  889875,  864461,
        837379,  766589,  748291,  740301,  739717,  721343,  703264,
        690480,  658175,  639500,  587866,  389322,  362897,  361028,
        352786,  291628,  269616,  261504,  236285,  226427,  226039,
        214165,  213

In [744]:
movies["Score"].unique()
movies["Score"]=movies["Score"].str.strip("0.e-0++f").str.replace(",",".").str.replace(":",".")
movies.loc[movies["Score"].str.contains("9."), "Score"] = "9.0"
movies.loc[movies["Score"].str.contains("8..8"), "Score"] = "8.8"
movies["Score"]=movies["Score"].astype(float)
movies["Score"].unique()

array([9. , 8.9, 8.8, 8.7, 8.6, 8.5, 8.4, 8.3, 8.2, 8.1, 8. , 7.9, 7.8,
       7.7, 7.6, 7.5, 7.4])